# Run full Granual Predictions

What needs to happen:
- ~~function to temporally and spatially interpolate MERRA2~~
- ~~function to read MERRA2 with interpolation~~
  - PS = surface_pressure
  - T10M = 10-meter_air_temperature (Andy will investigate)
  - TO3 = total_column_ozone
  - TQV = total_precipitable_water_vapor
- ~~function to read VNP02MOD~~
  - in group "observation_data"
    - M14 (for center wavelengths at 8500 nm)
    - M15 (for center wavelengths at 10800 nm)
    - M16 (for center wavelengths at 12000 nm)
- ~~function to read VNP03MOD~~
  - in group "geolocation_data"
    - sensor_azimuth
    - sensor_zenith
    - solar_azimuth
    - solar_zenith
- combine above for model inputs
- check if there are any transformations that need to be made on the inputs before sening them though the model
- load the saved model (see tensorflow.keras.models.load_model as in evaluate.ipynb)
- call the saved model's "predict" method on the combined inputs

## Setup

In [1]:
from pathlib import Path

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from hot_dust import preprocess, DATADIR

In [2]:
open_vnp02 = xr.open_dataset(
    "data/granules/VNP02MOD.A2020168.1448.002.2021127084950.nc",
    group="observation_data",
    mask_and_scale=False,
)

# Access the relavant variables
vnp02_variables = open_vnp02[
    [
        "M14",
        "M14_brightness_temperature_lut",
        "M15",
        "M15_brightness_temperature_lut",
        "M16",
        "M16_brightness_temperature_lut",
    ]
]

# call xr where and fill values less than oe equal to 65527 
vnp02_variables =  xr.where(vnp02_variables >= 65527, np.nan, vnp02_variables) #TODO stack, drop nan (drop same indices from others), convert to int, unstack  

# Create a mask for the NaN values and fill them   
nan_mask = xr.where(np.isnan(vnp02_variables), True, False)
vnp02_variables = vnp02_variables.where(~nan_mask, other=-999.99)

vnp02_variables

# xr stack and unstack 3D to 2D

<xarray.Dataset>
Dimensions:                         (number_of_lines: 3232,
                                     number_of_pixels: 3200,
                                     number_of_LUT_values: 65536)
Dimensions without coordinates: number_of_lines, number_of_pixels,
                                number_of_LUT_values
Data variables:
    M14                             (number_of_lines, number_of_pixels) float64 ...
    M14_brightness_temperature_lut  (number_of_LUT_values) float32 120.0 ... ...
    M15                             (number_of_lines, number_of_pixels) float64 ...
    M15_brightness_temperature_lut  (number_of_LUT_values) float32 111.0 ... ...
    M16                             (number_of_lines, number_of_pixels) float64 ...
    M16_brightness_temperature_lut  (number_of_LUT_values) float32 103.0 ... ...

In [3]:
open_vnp03 = xr.open_dataset(
    "data/granules/VNP03MOD.A2020168.1448.002.2021125194020.nc",
    group="geolocation_data",
)

# Access the relavant variables
vnp03_variables = open_vnp03[
    ["sensor_azimuth", "sensor_zenith", "solar_azimuth", "solar_zenith"]
]

In [4]:
# Merge the model imputs
vnp02_vnp03 = xr.merge([vnp02_variables, vnp03_variables])

# Stack the model inputs (did this in the blocks of code for merra)
vnp02_vnp03.stack(stack_dim=('number_of_lines', 'number_of_pixels'))


# Drop the NaN values (didn't work)
#vnp02_vnp03 = vnp02_vnp03.dropna(dim= 'number_of_lines') 
#vnp02_vnp03 = vnp02_vnp03.dropna(dim= 'number_of_pixels') 

# Convert temperature variables to integers
vnp02_vnp03["M14"] = vnp02_vnp03["M14"].astype(int)
vnp02_vnp03["M15"] = vnp02_vnp03["M15"].astype(int)
vnp02_vnp03["M16"] = vnp02_vnp03["M16"].astype(int)

# Multiply them within the xarray 
vnp02_vnp03["M14"] = vnp02_vnp03["M14_brightness_temperature_lut"][vnp02_vnp03["M14"]]
vnp02_vnp03["M15"] = vnp02_vnp03["M14_brightness_temperature_lut"][vnp02_vnp03["M15"]]
vnp02_vnp03["M16"] = vnp02_vnp03["M14_brightness_temperature_lut"][vnp02_vnp03["M16"]]  

vnp02_vnp03

<xarray.Dataset>
Dimensions:                         (number_of_lines: 3232,
                                     number_of_pixels: 3200,
                                     number_of_LUT_values: 65536)
Dimensions without coordinates: number_of_lines, number_of_pixels,
                                number_of_LUT_values
Data variables:
    M14                             (number_of_lines, number_of_pixels) float32 ...
    M14_brightness_temperature_lut  (number_of_LUT_values) float32 120.0 ... ...
    M15                             (number_of_lines, number_of_pixels) float32 ...
    M15_brightness_temperature_lut  (number_of_LUT_values) float32 111.0 ... ...
    M16                             (number_of_lines, number_of_pixels) float32 ...
    M16_brightness_temperature_lut  (number_of_LUT_values) float32 103.0 ... ...
    sensor_azimuth                  (number_of_lines, number_of_pixels) float32 ...
    sensor_zenith                   (number_of_lines, number_of_pixels) float32 ...
    solar_azimuth                   (number_of_lines, number_of_pixels) float32 ...
    solar_zenith                    (number_of_lines, number_of_pixels) float32 ...

In [14]:
# Access the relavant variables
M14_ds = vnp02_vnp03["M14"]  # thermal infared wave length
M14_BTL_ds = vnp02_vnp03["M14_brightness_temperature_lut"]
M15_ds = vnp02_vnp03["M15"]  # thermal infared wave length
M15_BTL_ds = vnp02_vnp03["M15_brightness_temperature_lut"]
M16_ds = vnp02_vnp03["M16"]  # thermal infared wave length
M16_BTL_ds = vnp02_vnp03["M16_brightness_temperature_lut"]

# Convert to brightness temperature in K
M14_scaled = vnp02_vnp03["M14_brightness_temperature_lut"][vnp02_vnp03["M14"]]
valid_min = vnp02_vnp03["M14_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M14_brightness_temperature_lut"].attrs["valid_max"]
M14_scaled = M14_scaled.where((M14_scaled <= valid_max) & (M14_scaled >= valid_min))

M15_scaled = vnp02_vnp03["M15_brightness_temperature_lut"][vnp02_vnp03["M15"]]
valid_min = vnp02_vnp03["M15_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M15_brightness_temperature_lut"].attrs["valid_max"]
M15_scaled = M15_scaled.where((M15_scaled <= valid_max) & (M15_scaled >= valid_min))

M16_scaled = vnp02_vnp03["M16_brightness_temperature_lut"][vnp02_vnp03["M16"]]
valid_min = vnp02_vnp03["M16_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M16_brightness_temperature_lut"].attrs["valid_max"]
M16_scaled = M16_scaled.where((M16_scaled <= valid_max) & (M16_scaled >= valid_min)) 

KeyError: 'M14_brightness_temperature_lut'

## Extract MERRA-2 Input Variables



In [6]:
(DATADIR / 'merra').exists() 
directory_path = (DATADIR / 'merra').glob('*.nc')  
list(directory_path)

[WindowsPath('c:/Users/micah/hot-dust/data/merra/VNP03MOD.A2020168.1448.002.2021125194020.nc'),
 WindowsPath('c:/Users/micah/hot-dust/data/merra/VNP03MOD.A2020169.0318.002.2021125194543.nc'),
 WindowsPath('c:/Users/micah/hot-dust/data/merra/VNP03MOD.A2020169.1430.002.2021125195004.nc'),
 WindowsPath('c:/Users/micah/hot-dust/data/merra/VNP03MOD.A2020170.0300.002.2021125195511.nc'),
 WindowsPath('c:/Users/micah/hot-dust/data/merra/VNP03MOD.A2020170.1412.002.2021125200321.nc')]

In [7]:
# Merge all the merra files 
directory_path = (DATADIR / 'merra'/ 'VNP03MOD.A2020168.1448.002.2021125194020.nc')
merra_variables = xr.open_dataset(directory_path) 
merra_variables = merra_variables.drop(['time', 'lon', 'lat']) # Drop the coordinates 

# Divide the MERRA2 pressure by 100 to get it in the right units
merra_variables['PS'] = merra_variables['PS']/100 

merra_variables

<xarray.Dataset>
Dimensions:  (number_of_lines: 3232, number_of_pixels: 3200)
Dimensions without coordinates: number_of_lines, number_of_pixels
Data variables:
    PS       (number_of_lines, number_of_pixels) float64 965.8 ... 1.024e+03
    TS       (number_of_lines, number_of_pixels) float64 ...
    TO3      (number_of_lines, number_of_pixels) float64 ...
    TQV      (number_of_lines, number_of_pixels) float64 ...

In [21]:
# drop LUT values  
#vnp02_vnp03 = vnp02_vnp03.drop_vars(['M14_brightness_temperature_lut', 'M15_brightness_temperature_lut', 'M16_brightness_temperature_lut'])  


# xr stack number of lines and pixels vnp03 and vnp02 make it 1D using the stack function
#variables_merged = xr.merge([vnp02_vnp03, merra_variables], compat='override') # Merge 1st, then stack
#variables_stacked = variables_merged.stack(dims_stacked = ('number_of_lines', 'number_of_pixels'))  

vnp02_vnp03

<xarray.Dataset>
Dimensions:                         (number_of_lines: 3232,
                                     number_of_pixels: 3200,
                                     number_of_LUT_values: 65536)
Dimensions without coordinates: number_of_lines, number_of_pixels,
                                number_of_LUT_values
Data variables:
    M14                             (number_of_lines, number_of_pixels) float32 ...
    M15                             (number_of_lines, number_of_pixels) float32 ...
    M15_brightness_temperature_lut  (number_of_LUT_values) float32 111.0 ... ...
    M16                             (number_of_lines, number_of_pixels) float32 ...
    M16_brightness_temperature_lut  (number_of_LUT_values) float32 103.0 ... ...
    sensor_azimuth                  (number_of_lines, number_of_pixels) float32 ...
    sensor_zenith                   (number_of_lines, number_of_pixels) float32 ...
    solar_azimuth                   (number_of_lines, number_of_pixels) float32 ...
    solar_zenith                    (number_of_lines, number_of_pixels) float32 ...

In [18]:
variables_stacked.to_array(dim='feature')

MemoryError: Unable to allocate 4.93 TiB for an array with shape (10342400, 65536) and data type float64

## 2D Plot of Input Variables

In [ ]:
# M14 Map (red)
plt.imshow(vnp02_vnp03["M14"], cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()  

In [ ]:
# M15 Map (green)
plt.imshow(M15_scaled, cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# M16 Map (blue)
plt.imshow(M16_scaled, cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

## 2D Plot of Predicted Dust Optical Thickness

## WIP / Scratch

In [ ]:
xr.open_dataset("data/rt_nn_irdust_training_data.nc")

In [ ]:
xr.open_dataset("data/granules/GMAO_MERRA2.20200616T140000.MET.nc")

True Color Map

In [ ]:
# Overlaying M14 (Red), M15 (Green), and M16 (Blue)
plt.imshow(M14_scaled, cmap="Reds", alpha=0.5)
plt.imshow(M15_scaled, cmap="Greens", alpha=0.5)
plt.imshow(M16_scaled, cmap="Blues", alpha=0.5)

# Colorbar and labels
cb = plt.colorbar(shrink=0.5)
plt.title('True Color Image')

# Show the plot
plt.show()